## The `FakeDataFrameBuilder` class
The heavy lifter for this assignment is named `FakeDataFrameBuilder`.  It resides in the file `faker_maker.py`.  It is designed to be used as the magic function `%%fakedata` which is demonstrated in this notebook.  Using it as a class instead of a magic function is also possible.  Create an instance of it as such:
```
from faker_maker import FakeDataFrameBuilder
cls = FakeDataFrameBuilder(text)
```
Where `text` is the multi-line text blob containing one or more dataframe definitions.  The worker function that parses the text blob and builds the dataframes is `parse` and is called like this:
```
cls.parse()
```
Access the results within the variable `dataframes`.  Since multiple dataframes might be built, this object is a dictionary of each dataframe indexed by the name.
```
persons = cls.dataframes['persons']
```
Below are a few notes on the structure and operation that may differ from the request and provide more clarity.

## An attempt at a more formal grammar

```
function_to_call  ::= <wordcharacters>
parameters        ::= "" | "(" ( wordcharacters | number ) ")"
as_name           ::= "" | "as" <whitespace> <wordcharacters>
column_name       ::= as_name | function_to_call
reference         ::= "" |  "[" number1, number2 "]"
unique_mark       ::= "" | "*"
column_definition ::= <function_to_call> <parameters> <whitespace> \
                      <as_name> <whitespace> <reference> <unique_mark>
df_sep            ::= "--" ("-"*)
df_definition     ::= <wordcharacters> <newline> <df_sep> <newline> \
                      (<column_definition>*) <newline> <newline>
language_spec     ::= <def_definition>*
```
Source: Prof. Christopher Brooks (brooksch@umich.edu)

### Table Definitions
The table defintion has been expanded from the request to include a row count.  The default remains `99` records if no row is provided but now a new value can be specified using the special syntax ` {#}` after the table name.  For example: 

```
purchases {30}
--------------
```

If tables are to have parent-child relationships and linked by a reference (see below), then the parent defintion should be provided first and the children after.

### Comments
The domain specific language invented for this problem was expanded to support comments similar to python using a `#`.  Comments are not supported throughout.  Examples where comments can be added:
 - After a table name (and optional row count)
 - After a field defintion
 - Before a field defintion (to comment out the field)


### Unique Values
The "unique" constraint is a bit ambiguous because let's say you have 2 columns (fields) that are defined as being unique within a dataset like in the example that was provided.  This could be interpreted in two ways: 

 1. The combination of the 2 values must be unique within the dataset
 2. Every value in the column must be unique
 
For this implementation, I found that the latter is more flexible in that it satisifies both of these points.  However, this comes with caveats that should be noted. The most obvious is a limitation of values.  For example, if you specify a column to contain a random number of length 2 and also unique, it is impossible to have a dataset with over 100 unique entries.  In fact, in how this is implemented, it is likely it would throw an exception much earlier due to how we limit the number times we look for a unique value.  Since we're using Faker to generate unique values, it should be kept in mind that there will be some limit to the number of unique values for each function.  For example, `last_name` might only generate 100 unique last names.  In my tests, I was getting duplicate values quite early on so its possible a unique set of last_names may randomly throw an error for a set of 99 unique last names.  It is recommeneded to read the Faker documentation for more information on what the limitations of each function are if you start getting errors about 'Maxed out unique checks'.

 - Future enhancement: Create a solution to #1 above.
 
### Parameters
All parameters must define the varable they are referencing in the parameter list.  This is necessary for how the parameter dictionary structure is built.  It's also good practice because it makes the parameter purpose much clearer.  For example:
```
random_number(digits=5) as customer_number
```

### References
This was expanded to allow for a comma delimited list of references and is demonstrated below.  The purpose is in the case that you want to drill down into a narrower and narrower set of values.  In the example below, not all persons will have purchases and not all purchases will have comments.  But, all comments should have one or more purchases and all purchases should have one person (since customer_number is unique on persons).  Here's how this would look:

```
persons
-------
random_number(digits=5) as customer_number [1]*
<other fields...>

purchases
-------
random_number(digits=5) as customer_number [1,2]
<other fields...>

comments
-------
random_number(digits=5) as customer_number [2]
<other fields...>
```

NOTE: The `function_name` is not actually used on child tables since the values are actually retrieved from the parent table.  In the above example, `purchases` and `comments` all get their values from `persons` so the `random_number(digits=5)` is actually not used or validated against.
 
### Repeat Data (Goal = 25% Duplicates)
This touches on the same vain as unique since we're doing some manual tricks to try to meet this 25% goal.  The solution I chose for this is to generate 75% of the values (note: this is controlled by the field `FakeDataFrameBuilder.repeat_threshold`) from Faker.  It then analyzes the uniqueness of the results.  If 25% or more duplicates already result, we just hit up the Faker for the remaining 25% (i.e. tough luck, go look at the other guy).  If it's >0% but <25%, then we salt it by hitting up the Faker again.

For example, if the uniqueness value comes back as 20% duplicates in the first 75% generated, it's pretty safe to say that the Faker is already doing a good job at getting close to our 25% limit.  So we'll get 20% more values from the faker.

For the remaining percent (or 25% if there were no unique values), we randomly sample from within our generated list.  This will obviously result in duplicates or possibly even triplicates or more.


### Seeding
I wanted a method to regenerate the same dataset so I implemented a `seed()` function.  This generally works but doesn't seem to work well with my random_number() generator for customer_number yet.  Still need to figure that one out...

 - Issue - `seed()` still generates random customer_numbers

The seed can now be set on the magic function call line using the syntax `seed=0`.

### Language
The language can also be defined on the magic function line using the syntax `lang=jp_JP`.
 
### Conclusion
This was created by Nicholas Miller (nmill@umich.edu) from designs provided above by Prof. Christopher Brooks (brooksch@umich.edu).  Submitted 2020-09-23.

In [1]:
import fakermaker

In [2]:
%load_ext fakermaker

In [3]:
%%fakedata seed=2, lang=jp_JP
# This is an improved version that supports comments
persons {10}
-----------------------
first_name  #This is a comment
last_name*  #Every lastname in the dataset will be unique
phone_number
#test('first',2=2,third=3,4)
random_number(digits=5) as customer_number [1]*

purchases     # I've also improved it to include an optional size
---------
isbn10
credit_card_full
random_number(digits=3) as price
random_number(digits=5) as customer_number [1,2]  # this reference says it's a child of persons and parent of comments

comments {2000}
---------
text(max_nb_chars=280) as comment
random_number(digits=5) as customer_number [2]

In [4]:
persons

,first_name,last_name,phone_number,customer_number
0,健一,渡辺,080-8233-0252,77063
1,和也,田中,080-7478-8577,53303
2,裕太,佐藤,84-0057-5668,40874
3,太一,池田,080-8858-8276,95804
4,学,中川,84-0057-5668,27239
5,真綾,山崎,090-9557-2678,64081
6,和也,岡田,080-7478-8577,67094
7,健一,石井,090-9877-3529,48050
8,結衣,山口,080-8858-8276,89683
9,健一,小林,090-7448-8889,81692


In [5]:
purchases

,isbn10,credit_card_full,price,customer_number
0,0-9773795-4-X,Discover\n篤司 渡辺\n6545396011225480 11/24\nCVC: ...,419,71554
1,0-16-742589-7,Discover\n修平 山田\n6011824928182051 12/21\nCVC: ...,642,66055
2,0-16-742589-7,Mastercard\n英樹 林\n5556393377176504 11/28\nCVV:...,609,6695
3,1-71670-499-5,Mastercard\n英樹 林\n5556393377176504 11/28\nCVV:...,747,94088
4,1-316-83499-9,JCB 16 digit\n陽子 山田\n3562231735314010 08/25\nC...,870,1801
...,...,...,...,...
94,1-84725-537-X,VISA 16 digit\n晃 吉田\n4245891286950679 08/27\nC...,720,58737
95,1-4487-4362-1,Mastercard\n加奈 高橋\n2276791667947086 07/29\nCVV...,17,6695
96,0-354-26541-5,VISA 13 digit\n直樹 伊藤\n4980066638411 03/27\nCVC...,759,89347
97,0-597-39924-7,JCB 15 digit\n桃子 青木\n213150843283371 11/21\nCV...,501,84299


In [6]:
comments

,comment,customer_number
0,クール見出し残るスペル自体ハードウェア偏差保持する。クルーフェミニスト管理する野球ない画面画...,97044
1,シュガー必要本質的な指名スマッシュ。溝スペル立派な販売叔父賞賛する。リハビリ符号クロス評議会...,63230
2,部隊君は偏差月。行進ダイヤモンド中央パーセント敵。コンペあなた自身バナー参加する脊椎。\n葉...,28553
3,月供給ボトル保証金差別する錯覚野球。職人じぶんの野球連続電池バス。偏差コンペ持ってる。\n移...,69121
4,デフォルト評議会ピックカレッジ彼女。スマッシュリンク尿意図。\n憲法探査数字埋め込む出演者賞...,19354
...,...,...
1995,今日人形再現するヒールストレージトーン。省略カラム意図ヒール倫理極端なバス軸。\n職人暖かい...,50392
1996,大統領ささやき証言する衝突編組文言見落とす。擁する〜転倒トレーナー行進高い敵。\nジャーナル...,99322
1997,ヒット奨励します私。溝副索引錯覚コンペブラケット。衝突デフォルト彼女鉱山バスケット。\nトス...,55440
1998,あった証言する行進人形。君はないあなた自身虐待ジャム。文言改善立派な。\n脊椎行進催眠術ささ...,13675


In [7]:
persons_cnt = len(persons)
purchases_cnt = len(purchases)
comments_cnt = len(comments)

repeat_text = '''Unique data checks:
 - persons.first_name {:.2f}%
 - persons.last_name {:.2f}%
 - persons.phone_number {:.2f}%
 - persons.customer_number {:.2f}%
 - purchases.isbn10 {:.2f}%
 - purchases.credit_card_full {:.2f}%
 - purchases.price {:.2f}%
 - purchases.customer_number {:.2f}%
 - comments.comment {:.2f}%
 - comments.customer_number {:.2f}%
 '''

print(repeat_text.format((1 - (len(persons['first_name'].unique()) / len(persons))) * 100,
                         (1 - (len(persons['last_name'].unique()) / len(persons))) * 100,
                         (1 - (len(persons['phone_number'].unique()) / len(persons))) * 100,
                         (1 - (len(persons['customer_number'].unique()) / len(persons))) * 100,
                         (1 - (len(purchases['isbn10'].unique()) / len(purchases))) * 100,
                         (1 - (len(purchases['credit_card_full'].unique()) / len(purchases))) * 100,
                         (1 - (len(purchases['price'].unique()) / len(purchases))) * 100,
                         (1 - (len(purchases['customer_number'].unique()) / len(purchases))) * 100,
                         (1 - (len(comments['comment'].unique()) / len(comments))) * 100,
                         (1 - (len(comments['customer_number'].unique()) / len(comments))) * 100))


Unique data checks:
 - persons.first_name 30.00%
 - persons.last_name 0.00%
 - persons.phone_number 30.00%
 - persons.customer_number 0.00%
 - purchases.isbn10 25.25%
 - purchases.credit_card_full 25.25%
 - purchases.price 25.25%
 - purchases.customer_number 25.25%
 - comments.comment 25.00%
 - comments.customer_number 25.00%
 
